In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
from fuzzywuzzy import fuzz, process
import utils as u

# Airport and country

In [13]:
url_passengers = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000001/~latest.csv?lang=en"
url_gm = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000002/~latest.csv"
url_op = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000003/~latest.csv"

In [14]:
data_t = pd.read_csv(u.get_data_from_API_call(url_passengers))

In [185]:
df = data_t.copy(deep=True)

In [186]:
df.drop(columns=df.columns[df.columns.str.endswith('#es')], inplace=True)

In [187]:
df['AEROPUERTO_ESCALA_CODE']

0               CV
1               CV
2               CV
3               CV
4               CV
            ...   
2963515    SE_ESMS
2963516    SE_ESMS
2963517    SE_ESMS
2963518    SE_ESMS
2963519    SE_ESMS
Name: AEROPUERTO_ESCALA_CODE, Length: 2963520, dtype: object

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963520 entries, 0 to 2963519
Data columns (total 16 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   SERVICIO_AEREO#en                object 
 1   SERVICIO_AEREO_CODE              object 
 2   MEDIDAS#en                       object 
 3   MEDIDAS_CODE                     object 
 4   MOVIMIENTO_AERONAVE#en           object 
 5   MOVIMIENTO_AERONAVE_CODE         object 
 6   TIME_PERIOD#en                   object 
 7   TIME_PERIOD_CODE                 object 
 8   AEROPUERTO_BASE#en               object 
 9   AEROPUERTO_BASE_CODE             object 
 10  AEROPUERTO_ESCALA#en             object 
 11  AEROPUERTO_ESCALA_CODE           object 
 12  OBS_VALUE                        float64
 13  ESTADO_OBSERVACION#en            object 
 14  ESTADO_OBSERVACION_CODE          object 
 15  CONFIDENCIALIDAD_OBSERVACION#en  float64
dtypes: float64(2), object(14)
memory usage: 361.8+ MB


Steps to getting the airports right:
1) Delete countries, autonomous communities and "Rest of"/"Remain of"
2) Join with airports from ourairports using word similitude or something like that

[Understanding _CODE from airports](https://www3.gobiernodecanarias.org/aplicaciones/appsistac/activos-semanticos/codelists/codelists/ISTAC/CL_AEROPUERTOS/01.004/detail)

I think I can do all I did (selecting only airports) with the API, with the "granularity" option, tomorrow ill check

Hello, it is tomorrow, it can not be done, I tried

In [189]:
# Extract countries
countries = df.loc[df['AEROPUERTO_ESCALA_CODE'].str.match(r'^[A-Z]{2}$', na=False), ['AEROPUERTO_ESCALA#en', 'AEROPUERTO_ESCALA_CODE']]
countries.drop_duplicates(inplace=True)
countries.rename({'AEROPUERTO_ESCALA#en': 'CountryName', 'AEROPUERTO_ESCALA_CODE': 'iso_country'}, inplace=True, axis=1)

In [190]:
countries

,CountryName,iso_country
0,Cabo Verde,CV
3360,Iceland,IS
9240,France,FR
15120,Gambia,GM
16240,Netherlands,NL
19320,Belgium,BE
21280,Morocco,MA
23520,Norway,NO
24360,Ukraine,UA
28840,Lithuania,LT


In [191]:
# Rest of/Remain of AEROPUERTO_ESCALA#en have "_O" at the end of their code
df_f = df.loc[~df['AEROPUERTO_ESCALA_CODE'].str.endswith('_O')]

# Delete entries with the whole country
df_f = df_f.loc[~df_f['AEROPUERTO_ESCALA_CODE'].str.match(r'^[A-Z]{2}$', na=False)]

# Delete autonomous communities (Their code is like ES[0-9][0-9]) 
df_f = df_f.loc[~df_f['AEROPUERTO_ESCALA_CODE'].str.match(r'^ES[0-9]{2}$', na=False)]

# Delete sum of entire island
df_f = df_f.loc[~df_f['AEROPUERTO_ESCALA_CODE'].str.match(r'^ES70[0-9]$', na=False)]

# Delete sum of all autonomous communities and sum of all islands
df_f = df_f.loc[~((df_f['AEROPUERTO_ESCALA_CODE'] == 'ES_XES70') | (df_f['AEROPUERTO_ESCALA_CODE'] == 'ES70') | (df_f['AEROPUERTO_ESCALA_CODE'] == 'FOREIGN'))]

In [192]:
df_f.loc[~df_f['AEROPUERTO_ESCALA#en'].str.endswith('Airport', na=False), 'AEROPUERTO_ESCALA#en'].unique()

array(['Trondheim Airport Vèrnes', 'Zaragoza Air Base',
       'Sandefjord Airport, Torp', 'Harstad/Narvik Airport, Evenes',
       'Stavanger Airport Sola', 'Amsterdam Airport Schiphol',
       'Václav Havel Airport Prague', 'Bergen Airport Flesland',
       'Moss Airport, Rygge'], dtype=object)

In [245]:
istac_airports = df_f[['AEROPUERTO_ESCALA#en', 'AEROPUERTO_ESCALA_CODE']].copy(deep=True)

In [246]:
istac_airports.drop_duplicates(inplace=True)

In [247]:
istac_airports

,AEROPUERTO_ESCALA#en,AEROPUERTO_ESCALA_CODE
280,Nouadhibou International Airport,MR_GQPP
560,Brussels Airport,BE_EBBR
840,Durham Tees Valley Airport,GB_EGNV
1120,Dublin Airport,IE_EIDW
1400,Växjö Kronoberg Airport,SE_ESMX
...,...,...
80640,Francisco de Sá Carneiro Airport,PT_LPPR
80920,Zweibrücken Airport,DE_EDRZ
81480,Vigo Airport,ES_LEVX
81760,Léopold Sédar Senghor International Airport,SN_GOOY


In [248]:
istac_airports['ident'] = istac_airports['AEROPUERTO_ESCALA_CODE'].str[3:]

In [249]:
airport_csv = pd.read_csv('airports.csv')

In [250]:
airport_csv = airport_csv.merge(countries, on='iso_country')

In [251]:
join = istac_airports.merge(airport_csv, on='ident', suffixes=("l", "r"))

In [252]:
join

,AEROPUERTO_ESCALA#en,AEROPUERTO_ESCALA_CODE,ident,id,type,name,latitude_deg,longitude_deg,elevation_ft,continent,...,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords,CountryName
0,Nouadhibou International Airport,MR_GQPP,GQPP,3138,medium_airport,Nouadhibou International Airport,20.933100,-17.030001,24.0,AF,...,Nouadhibou,yes,GQPP,NDB,GQPP,NaN,NaN,https://en.wikipedia.org/wiki/Nouadhibou_Inter...,NaN,Mauritania
1,Brussels Airport,BE_EBBR,EBBR,2155,large_airport,Brussels Airport,50.901402,4.484440,175.0,EU,...,Zaventem,yes,EBBR,BRU,EBBR,NaN,http://www.brusselsairport.be/en/,https://en.wikipedia.org/wiki/Brussels_Airport,"Brussels National, Zaventem, EBMB",Belgium
2,Durham Tees Valley Airport,GB_EGNV,EGNV,2449,medium_airport,Teesside International Airport,54.509201,-1.429410,120.0,EU,...,"Darlington, Durham",yes,EGNV,MME,EGNV,NaN,https://www.teessideinternational.com/,https://en.wikipedia.org/wiki/Teesside_Interna...,"Durham Tees Valley Airport, RAF Middleton St G...",United Kingdom of Great Britain and Northern I...
3,Dublin Airport,IE_EIDW,EIDW,2533,large_airport,Dublin Airport,53.428713,-6.262121,242.0,EU,...,Dublin,yes,EIDW,DUB,EIDW,NaN,http://www.dublinairport.com/,https://en.wikipedia.org/wiki/Dublin_Airport,Aerfort Bhaile Átha Cliath,Ireland
4,Växjö Kronoberg Airport,SE_ESMX,ESMX,2672,medium_airport,Växjö Kronoberg Airport,56.929100,14.728000,610.0,EU,...,Växjö,yes,ESMX,VXO,ESMX,NaN,NaN,https://en.wikipedia.org/wiki/V%C3%A4xj%C3%B6_...,NaN,Sweden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Francisco de Sá Carneiro Airport,PT_LPPR,LPPR,4459,large_airport,Francisco de Sá Carneiro Airport,41.248100,-8.681390,228.0,EU,...,Porto,yes,LPPR,OPO,LPPR,NaN,http://www.ana.pt/en-US/Aeroportos/porto/Porto...,https://en.wikipedia.org/wiki/Francisco_S%C3%A...,NaN,Portugal
190,Zweibrücken Airport,DE_EDRZ,EDRZ,2269,small_airport,Zweibrücken Airport,49.209400,7.400560,1132.0,EU,...,Zweibrücken,no,EDRZ,ZQW,EDRZ,NaN,http://www.edrz-airport.de,https://en.wikipedia.org/wiki/Zweibr%C3%BCcken...,NaN,Germany
191,Vigo Airport,ES_LEVX,LEVX,4046,medium_airport,Vigo Airport,42.231800,-8.626770,856.0,EU,...,Vigo,yes,LEVX,VGO,LEVX,NaN,https://www.aena.es/en/vigo.html,https://en.wikipedia.org/wiki/Vigo-Peinador_Ai...,NaN,Spain
192,Léopold Sédar Senghor International Airport,SN_GOOY,GOOY,3125,medium_airport,Léopold Sédar Senghor International Airport,14.739700,-17.490200,85.0,AF,...,Dakar,no,GOOY,DKR,GOOY,NaN,http://www.aeroportdakar.com/,https://en.wikipedia.org/wiki/Dakar-Yoff-L%C3%...,Yoff,Senegal


Only two missing airports, I can input them by hand

In [244]:
print(f"Mising airports: {len(istac_airports['AEROPUERTO_ESCALA#en'].unique()) - len(join['AEROPUERTO_ESCALA#en'].unique())}")
missing_airports = set(istac_airports['AEROPUERTO_ESCALA#en'].unique()) - set(join['AEROPUERTO_ESCALA#en'].unique())
print(f"Missing airports: {missing_airports}")

Mising airports: 2
Missing airports: {'Berlin-Tegel Airport', 'Robin Hood Doncaster Sheffield Airport'}


In [257]:
result_df = join[['AEROPUERTO_ESCALA#en', 'AEROPUERTO_ESCALA_CODE', 'latitude_deg', 'longitude_deg','iso_country', 'CountryName']].copy(deep=True)
result_df.rename({'iso_country': 'CountryCode', 'AEROPUERTO_ESCALA#en': 'AirportName', 
                  'latitude_deg': 'Latitude', 'longitude_deg': 'Longitude', 
                  'country': 'CountryName', 'AEROPUERTO_ESCALA_CODE': 'AirportCode'}, axis=1, inplace=True)

In [258]:
result_df['AirportId'] = result_df.index

In [259]:
result_df.to_csv('Airport.csv', index=False)
result_df

,AirportName,AirportCode,Latitude,Longitude,CountryCode,CountryName,AirportId
0,Nouadhibou International Airport,MR_GQPP,20.933100,-17.030001,MR,Mauritania,0
1,Brussels Airport,BE_EBBR,50.901402,4.484440,BE,Belgium,1
2,Durham Tees Valley Airport,GB_EGNV,54.509201,-1.429410,GB,United Kingdom of Great Britain and Northern I...,2
3,Dublin Airport,IE_EIDW,53.428713,-6.262121,IE,Ireland,3
4,Växjö Kronoberg Airport,SE_ESMX,56.929100,14.728000,SE,Sweden,4
...,...,...,...,...,...,...,...
189,Francisco de Sá Carneiro Airport,PT_LPPR,41.248100,-8.681390,PT,Portugal,189
190,Zweibrücken Airport,DE_EDRZ,49.209400,7.400560,DE,Germany,190
191,Vigo Airport,ES_LEVX,42.231800,-8.626770,ES,Spain,191
192,Léopold Sédar Senghor International Airport,SN_GOOY,14.739700,-17.490200,SN,Senegal,192


# Territory, AircraftMovement, AirService

In [260]:
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000013/~latest.csv?lang=en"

In [261]:
data = pd.read_csv(u.get_data_from_API_call(url))

In [262]:
data

,MEDIDAS#en,MEDIDAS#es,MEDIDAS_CODE,TERRITORIO#en,TERRITORIO#es,TERRITORIO_CODE,AEROPUERTO_ESCALA#en,AEROPUERTO_ESCALA#es,AEROPUERTO_ESCALA_CODE,MOVIMIENTO_AERONAVE#en,...,SERVICIO_AEREO_CODE,TIME_PERIOD#en,TIME_PERIOD#es,TIME_PERIOD_CODE,OBS_VALUE,ESTADO_OBSERVACION#en,ESTADO_OBSERVACION#es,ESTADO_OBSERVACION_CODE,CONFIDENCIALIDAD_OBSERVACION#en,CONFIDENCIALIDAD_OBSERVACION#es
0,Passengers,Pasajeros,PASAJEROS,Canary Islands,Canarias,ES70,United Kingdom of Great Britain and Northern I...,Reino Unido,GB,Arrival,...,COMMERCIAL,01/2004,01/2004,2004-M01,317378.0,Normal value,Valor normal,A,NaN,NaN
1,Passengers,Pasajeros,PASAJEROS,Canary Islands,Canarias,ES70,United Kingdom of Great Britain and Northern I...,Reino Unido,GB,Arrival,...,COMMERCIAL,02/2004,02/2004,2004-M02,320960.0,Normal value,Valor normal,A,NaN,NaN
2,Passengers,Pasajeros,PASAJEROS,Canary Islands,Canarias,ES70,United Kingdom of Great Britain and Northern I...,Reino Unido,GB,Arrival,...,COMMERCIAL,06/2010,06/2010,2010-M06,251362.0,Normal value,Valor normal,A,NaN,NaN
3,Passengers,Pasajeros,PASAJEROS,Canary Islands,Canarias,ES70,United Kingdom of Great Britain and Northern I...,Reino Unido,GB,Arrival,...,COMMERCIAL,07/2010,07/2010,2010-M07,296367.0,Normal value,Valor normal,A,NaN,NaN
4,Passengers,Pasajeros,PASAJEROS,Canary Islands,Canarias,ES70,United Kingdom of Great Britain and Northern I...,Reino Unido,GB,Arrival,...,COMMERCIAL,08/2010,08/2010,2010-M08,287040.0,Normal value,Valor normal,A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215035,Passengers,Pasajeros,PASAJEROS,El Hierro,El Hierro,ES703,Germany,Alemania,DE,Total,...,SCHEDULED,01/2010,01/2010,2010-M01,NaN,NaN,NaN,NaN,NaN,NaN
215036,Passengers,Pasajeros,PASAJEROS,El Hierro,El Hierro,ES703,Germany,Alemania,DE,Total,...,SCHEDULED,02/2010,02/2010,2010-M02,NaN,NaN,NaN,NaN,NaN,NaN
215037,Passengers,Pasajeros,PASAJEROS,El Hierro,El Hierro,ES703,Germany,Alemania,DE,Total,...,SCHEDULED,03/2010,03/2010,2010-M03,NaN,NaN,NaN,NaN,NaN,NaN
215038,Passengers,Pasajeros,PASAJEROS,El Hierro,El Hierro,ES703,Germany,Alemania,DE,Total,...,SCHEDULED,04/2010,04/2010,2010-M04,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
df = data.copy(deep=True)

In [264]:
df['TERRITORIO#en'].unique()

array(['Canary Islands', 'Lanzarote', 'Fuerteventura', 'Gran Canaria',
       'Tenerife', 'La Gomera', 'La Palma', 'El Hierro'], dtype=object)

In [265]:
df['AEROPUERTO_ESCALA#en'].unique()

array(['United Kingdom of Great Britain and Northern Ireland', 'Foreign',
       'Canary Islands', 'Foreign and Spain (Canary Islands excluded)',
       'Spain (Canary Islands excluded)', 'Spain', 'Total', 'Germany'],
      dtype=object)